### Install Pacakages

In [2]:
!pip -q install unsloth accelerate bitsandbytes transformers datasets
!pip -q install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 131.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q streamlit pyngrok

In [1]:
import os

os.makedirs('cot_dataset', exist_ok=False)

### Streamlit Application

In [ ]:
%%writefile app.py
# =====================================================
# STREAMLIT MCQ GENERATOR (DESIGN-TIME REASONING + SLOWER STREAMING RAW OUTPUT)
# =====================================================

import streamlit as st
import json
import torch
import re
import time
from unsloth import FastLanguageModel

# ===========================
# CONFIG
# ===========================
MODEL_PATH = "/content/phi-mcq-skill-exp-cot-dpo/final_model"

st.set_page_config(page_title="MCQ Generator", layout="wide")
st.title("🧠 MCQ Generator - Reasoning")

# ===========================
# SESSION STATE
# ===========================
if "results" not in st.session_state:
    st.session_state.results = []

# ===========================
# MODEL LOAD
# ===========================
@st.cache_resource
def load_model():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MODEL_PATH,
        max_seq_length=1536,
        load_in_4bit=True,
        dtype=torch.bfloat16
    )
    model.eval()
    return model, tokenizer

model, tokenizer = load_model()

# ===========================
# PROMPT BUILDER
# ===========================
def build_inference_prompt(skill: str, experience: str) -> str:
    system_prompt = (
        "You are an expert MCQ generator.\n"
        "You explain your reasoning in clear instructional steps.\n"
        "Do NOT reveal internal thoughts.\n"
        "Focus on concepts and option elimination."
    )

    user_prompt = f"""
Generate exactly ONE multiple-choice question.

Skill: {skill}
Experience Level: {experience}

Output format:

[REASONING]
Step 1 – ...
Step 2 – ...
Step 3 – ...
Step 4 – ...

[MCQ_JSON]
{{ ... }}
""".strip()

    return f"<|system|>\n{system_prompt}<|end|>\n<|user|>\n{user_prompt}<|end|>\n<|assistant|>\n"

# ===========================
# JSON EXTRACTOR
# ===========================
def extract_json(text: str) -> str | None:
    text = re.sub(r"```json|```", "", text).strip()
    stack, start = [], None
    for i, ch in enumerate(text):
        if ch == "{":
            if not stack:
                start = i
            stack.append(ch)
        elif ch == "}":
            if stack:
                stack.pop()
                if not stack and start is not None:
                    return text[start:i + 1]
    return None

# ===========================
# SYNTHETIC DESIGN-TIME REASONING
# ===========================
def generate_design_reasoning(mcq_data: dict, skill: str, experience: str) -> str:
    question = mcq_data.get("text", "")
    options = mcq_data.get("options", {})
    correct = mcq_data.get("correct_answer", "")
    explanation = mcq_data.get("explanation", "")

    correct_text = options.get(correct, "")
    distractors = [v for k, v in options.items() if k != correct]

    steps = [
        f"Step 1 – Topic Selection\nSelected a core '{skill}' concept suitable for {experience}.",
        f"Step 2 – Question Crafting\n\"{question}\""
    ]

    if distractors:
        distractor_desc = "Step 3 – Distractor Design\n"
        for i, _ in enumerate(distractors[:3], 1):
            distractor_desc += f"- Distractor {i}: Plausible misconception.\n"
        steps.append(distractor_desc.strip())

    steps.append(
        f"Step 4 – Correct Option Validation\n"
        f"Option {correct} (\"{correct_text}\") is correct because {explanation}"
    )

    return "\n\n".join(steps)

# ===========================
# MCQ GENERATION WITH SLOW STREAMING
# ===========================
@torch.inference_mode()
def generate_mcq_streaming(skill: str, experience: str, placeholder=None):
    """
    Generate MCQ and simulate slower streaming raw output.
    """
    prompt = build_inference_prompt(skill, experience)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=850,
        do_sample=True,
        temperature=0.8,
        top_p=0.92,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=[
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|end|>")
        ]
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Simulate slower streaming
    if placeholder:
        placeholder.markdown("```\n\n```")
        chunk_size = 20  # smaller chunks for slower streaming
        for i in range(0, len(decoded), chunk_size):
            placeholder.markdown(f"```\n{decoded[:i + chunk_size]}\n```")
            time.sleep(0.12)  # slower delay for visible streaming

    # Extract MCQ JSON
    r_idx = decoded.rfind("[REASONING]")
    j_idx = decoded.rfind("[MCQ_JSON]")

    mcq_data = {}
    if r_idx != -1 and j_idx != -1:
        json_text = extract_json(decoded[j_idx + len("[MCQ_JSON]"):])
        if json_text:
            try:
                mcq_data = json.loads(json_text)
            except:
                mcq_data = {}

    mcq_item = {}
    if isinstance(mcq_data, dict):
        if "mcq" in mcq_data:
            if isinstance(mcq_data["mcq"], list) and mcq_data["mcq"]:
                mcq_item = mcq_data["mcq"][0]
            elif isinstance(mcq_data["mcq"], dict):
                mcq_item = mcq_data["mcq"]

    reasoning = (
        generate_design_reasoning(mcq_item, skill, experience)
        if mcq_item else "⚠️ Failed to generate valid MCQ JSON."
    )

    return reasoning, mcq_item, decoded

# ===========================
# STREAMLIT UI
# ===========================
skill_input = st.text_input("Skill(s)", "Cloud Services, AWS")
experience = st.selectbox(
    "Experience Level",
    ["0-2 years", "2-5 years", "5-8 years", "8-12 years", "12-15 years", "15+ years"]
)
num_mcqs = st.number_input("Number of MCQs", 1, 10, 1)

# ===========================
# GENERATE BUTTON
# ===========================
if st.button("🚀 Generate MCQs"):
    st.session_state.results = []

    skills = [s.strip() for s in skill_input.split(",") if s.strip()]

    for i in range(num_mcqs):
        skill = skills[i % len(skills)]
        with st.spinner(f"Generating MCQ {i+1} for {skill}..."):
            raw_placeholder = st.empty()  # placeholder for streaming
            reasoning, mcq, raw = generate_mcq_streaming(skill, experience, raw_placeholder)

        st.session_state.results.append({
            "skill": skill,
            "reasoning": reasoning,
            "mcq": mcq,
            "raw": raw
        })

# ===========================
# RENDER RESULTS (AFTER RUN)
# ===========================
for idx, res in enumerate(st.session_state.results, 1):
    st.divider()
    st.subheader(f"MCQ {idx} | {res['skill']}")

    with st.expander("🧠 Design-Time Reasoning"):
        st.markdown(res["reasoning"])

    mcq = res["mcq"]
    st.markdown(f"### 📘 Question\n{mcq.get('text','')}")

    for k, v in mcq.get("options", {}).items():
        st.markdown(f"- **{k}.** {v}")

    st.success(f"✅ Correct Answer: {mcq.get('correct_answer','')}")
    st.info(f"💡 Explanation: {mcq.get('explanation','')}")

    # RAW MODEL OUTPUT DROPDOWN
    with st.expander("📄 Raw Model Output"):
        st.code(res["raw"])

In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!./cloudflared-linux-amd64 tunnel --url http://localhost:8501